# Please note that you need requests package to run this notebook

In [ ]:
import requests
import json
import pprint

server_port = 8000

# Get OAuth2 Access Token specifying existing taskgroup_uuid

In [ ]:
arguments = {
             'client_id': '8c96bf8cea26fa555fa8',
             'client_secret': '4fd1f508b7b03fba6509da4c193157d7a2b20838',
             'grant_type': 'password',
             'username': 'admin',
             'password': 'admin',
             'taskgroup': '5dw2x3jfkftxue5a5izw6yiplbbn4dlo'
             }
r = requests.post(
                  'http://127.0.0.1:{server_port}/oauth2/access_token/'.format(server_port=server_port), 
                  data=arguments
                  )
pprint.pprint(r.json())
access_token = json.loads(r.text)['access_token']

# Get EHR DBService status

In [ ]:
payload = {'access_token': access_token}
r = requests.get(
    'http://127.0.0.1:{server_port}/medicalrecords/ehr/status/'.format(server_port=server_port), 
    params=payload)
print r.text

# Create patient record

In [ ]:
payload = {'access_token': access_token, 'demographic_uuid': 'abc001'}

r = requests.post(
    'http://127.0.0.1:{server_port}/medicalrecords/patients/'.format(server_port=server_port), 
    params=payload
)
pprint.pprint(r.json())
data = json.loads(r.text)
patient_id = data['patient']['uuid']
print 'patient id: %s' % patient_id

# Create the EHR for the patient

In [ ]:
payload = {'access_token': access_token}

r = requests.post(
    'http://127.0.0.1:{server_port}/medicalrecords/ehr/{patient_id}/'.
    format(server_port=server_port, patient_id=patient_id), 
    params=payload
)
pprint.pprint(r.json())


# Add a record to the EHR

In [ ]:
params = {'access_token': access_token}

ehr_data = {
  "archetype_class": "openEHR.TEST-EVALUATION.v1",
  "archetype_details": {
    "at0001": "val1",
    "at0002": "val2"
  }
}

r = requests.post(
    'http://127.0.0.1:{server_port}/medicalrecords/ehr/{patient_id}/records/'.
        format(server_port=server_port, patient_id=patient_id), 
    params=params, json=ehr_data
)
pprint.pprint(r.json())

# Check new EHR entry for current medicalrecord patient

In [ ]:
payload = {'access_token': access_token}
r = requests.get(
    'http://127.0.0.1:{server_port}/medicalrecords/ehr/{patient_id}/'.
    format(server_port=server_port, patient_id=patient_id), 
    params=payload
)
try:
    data = json.loads(r.text)
except ValueError:
    print r.text
else:
    print 'number of records: %s ' % len(data['record']['ehr_records'])
    pprint.pprint(data)
    record_id = data['record']['ehr_records'][-1]['record_id']

# Get an ehr record

In [ ]:
payload = {'access_token': access_token}
r = requests.get(
    'http://127.0.0.1:{server_port}/medicalrecords/ehr/{patient_id}/records/{record_id}/'.
    format(server_port=server_port, patient_id=patient_id, record_id=record_id), 
    params=payload
)
pprint.pprint(r.json())

# Delete a record from the ehr

In [ ]:
payload = {'access_token': access_token}
r = requests.delete(
    'http://127.0.0.1:{server_port}/medicalrecords/ehr/{patient_id}/records/{record_id}/'.
        format(server_port=server_port, patient_id=patient_id, record_id=record_id), 
    params=payload)

pprint.pprint(r.json())

# Delete ehr patient record

In [ ]:
r = requests.delete(
    'http://127.0.0.1:{server_port}/medicalrecords/ehr/{patient_id}/'.
    format(server_port=server_port, patient_id=patient_id), 
params=payload, data={})

pprint.pprint(r.json())